<a href="https://colab.research.google.com/github/TetianaZabolotko/MLtrainings/blob/main/Kaggle_obesity_risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Your Goal: The goal of this competition is to use various factors
# to predict obesity risk in individuals, which is related to cardiovascular disease.
# Good luck!

import pandas as pd
import numpy as np

import matplotlib as plt

In [3]:
df_train = pd.read_csv('/home/yen/Comp_Vision/pet_projects/classification/Obesity_risk/train.csv')
df_test = pd.read_csv('/home/yen/Comp_Vision/pet_projects/classification/Obesity_risk/test.csv')
df_test = pd.read_csv('/home/yen/Comp_Vision/pet_projects/classification/Obesity_risk/sample_submission.csv')

print(f'The train dataset has {df_train.shape[0]}, the testset has {df_test.shape[0]}')

FileNotFoundError: [Errno 2] No such file or directory: '/home/yen/Comp_Vision/pet_projects/classification/Obesity_risk/train.csv'

In [4]:
display('Train:', df_train.head(10))
display('Test:',df_test.head())

NameError: name 'df_train' is not defined

In [5]:
df_train.describe().to_csv('output.csv', index=False)

NameError: name 'df_train' is not defined

In [6]:
df_train.info

NameError: name 'df_train' is not defined

In [ ]:
from tabulate import tabulate
data = [
    ["FAVC", "Frequent consumption of high caloric food"],
    ["FCVC", "Frequency of consumption of vegetables"],
    ["NCP", "Number of main meals"],
    ["CAEC", "Consumption of food between meals"],
    ["CH20", "Consumption of water daily"],
    ["CALC", "Consumption of alcohol"],
    ["SCC", "Calories consumption monitoring"],
    ["FAF", "Physical activity frequency"],
    ["TUE", "Time using technology devices"],
    ["MTRANS", "Transportation used"]
]
headers = ["Abbreviation", "Full Form"]

table = tabulate(data, headers, tablefmt="pipe")
print(table)

In [ ]:
def summary(df):
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['Duplicate'] = df.duplicated().sum()
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['avg'] = desc['mean'].values
    summ['std dev'] = desc['std'].values
    summ['top value'] = desc['top'].values
    summ['Freq'] = desc['freq'].values

    return summ

In [ ]:
summary(df_train.drop(columns=["id"])).style.background_gradient()